In [37]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [31]:
spatial = pd.read_csv('data/shortest_paths.csv')
social = pd.read_csv('data/social_network.csv',index_col=0)

In [32]:
spatial["source"] = spatial["source"].str.normalize("NFD").str.encode('ascii', 'ignore')\
    .str.decode("utf-8")
spatial["target"] = spatial["target"].str.normalize("NFD").str.encode('ascii', 'ignore')\
    .str.decode("utf-8")

In [33]:
social2 = social.copy()
social2["target"] = social["source"]
social2["source"] = social["target"]

social = pd.merge(social,social2,on=["year","source","target"])
social["w"] = social["w_x"] + social["w_y"]
del social["w_x"], social["w_y"]

In [95]:
final = pd.merge(spatial,social,how="left")
final["year_total"] = final["year"].map(final.groupby("year")["w"].sum())
final.sort_values(by=["source","target","year"],inplace=True)
final["diff"] = [0] + list(np.diff(final["travel_time"]))

for i in range(len(final.index)):
    if final["source"].iloc[i] == final["source"].iloc[i-1] and\
    final["target"].iloc[i] == final["target"].iloc[i-1]:
        pass
    else:
        final["diff"].iloc[i]=0

In [150]:
social_timelines = final.pivot_table(index=["source","target"],columns="year",values="w").fillna(0)
# social_timelines["total"] = final.pivot_table(index=["source","target"],columns="year",values="w").fillna(0).sum(axis=1)
# social_timelines = social_timelines.div(social_timelines["total"],axis='rows')
# del social_timelines["total"]

In [151]:
spatial_timelines = final.pivot_table(index=["source","target"],columns="year",values="travel_time").fillna(0).diff(axis=1).fillna(0)
# spatial_timelines.cumsum(axis=1)

In [152]:
social_timelines

year                         2002  2003  2004  2005  2006  2007  2008  2009  \
source       target                                                           
Bekescsaba   Gyor             0.0   0.0   0.0   0.0  36.0  45.0  19.0   0.0   
             Miskolc          0.0   0.0   0.0   4.0  23.0  34.0  13.0   0.0   
             Nyiregyhaza      0.0   0.0   0.0   0.0  21.0  23.0  13.0   2.0   
             Szekesfehervar   0.0   0.0   0.0   6.0  19.0  24.0   5.0   9.0   
             Veszprem         0.0   0.0   0.0   0.0  12.0  20.0   9.0   3.0   
...                           ...   ...   ...   ...   ...   ...   ...   ...   
Zalaegerszeg Szeged           0.0   0.0   0.0  14.0  53.0  24.0   7.0  11.0   
             Szekesfehervar   0.0   0.0   0.0   7.0  53.0  28.0  14.0  16.0   
             Szekszard        0.0   0.0   0.0   3.0   5.0  11.0   4.0   0.0   
             Tatabanya        0.0   0.0   0.0   3.0   7.0  12.0   3.0   8.0   
             Veszprem         0.0   0.0   2.0  12.0  63.0  52.0   9.0  12.0   

year                         2010  2011  2012  
source       target                            
Bekescsaba   Gyor            13.0   0.0   0.0  
             Miskolc          7.0   0.0   0.0  
             Nyiregyhaza      0.0   0.0   0.0  
             Szekesfehervar   0.0   0.0   0.0  
             Veszprem         0.0   0.0   0.0  
...                           ...   ...   ...  
Zalaegerszeg Szeged           6.0   0.0   0.0  
             Szekesfehervar   2.0   0.0   0.0  
             Szekszard        3.0   0.0   0.0  
             Tatabanya        0.0   0.0   0.0  
             Veszprem        13.0   0.0   0.0  

[171 rows x 11 columns]

In [154]:
spatial_timelines.cumsum(axis=1)

year                         2002  2003      2004      2005      2006  \
source       target                                                     
Bekescsaba   Gyor             0.0   0.0  0.000000  0.000000  0.000000   
             Miskolc          0.0   0.0 -0.108609 -0.108609 -0.558268   
             Nyiregyhaza      0.0   0.0  0.000000  0.000000  0.000000   
             Szekesfehervar   0.0   0.0  0.000000  0.000000  0.000000   
             Veszprem         0.0   0.0  0.000000  0.000000  0.000000   
...                           ...   ...       ...       ...       ...   
Zalaegerszeg Szeged           0.0   0.0  0.000000  0.000000 -0.824858   
             Szekesfehervar   0.0   0.0  0.000000  0.000000 -0.146245   
             Szekszard        0.0   0.0  0.000000  0.000000 -0.358691   
             Tatabanya        0.0   0.0  0.000000  0.000000  0.000000   
             Veszprem         0.0   0.0  0.000000  0.000000  0.000000   

year                             2007      2008      2009      2010      2011  \
source       target                                                             
Bekescsaba   Gyor            0.000000  0.000000  0.000000  0.000000  0.000000   
             Miskolc        -0.558268 -0.558268 -0.558268 -0.558268 -0.558268   
             Nyiregyhaza     0.000000  0.000000  0.000000  0.000000  0.000000   
             Szekesfehervar  0.000000  0.000000  0.000000  0.000000  0.000000   
             Veszprem        0.000000  0.000000  0.000000  0.000000  0.000000   
...                               ...       ...       ...       ...       ...   
Zalaegerszeg Szeged         -1.032718 -1.032718 -1.032718 -1.032718 -1.079081   
             Szekesfehervar -0.351365 -0.351365 -0.351365 -0.351365 -0.351365   
             Szekszard      -0.559632 -0.559632 -0.559632 -0.559632 -0.559632   
             Tatabanya       0.000000  0.000000  0.000000  0.000000  0.000000   
             Veszprem        0.000000  0.000000  0.000000  0.000000  0.000000   

year                             2012  
source       target                    
Bekescsaba   Gyor            0.000000  
             Miskolc        -0.558268  
             Nyiregyhaza     0.000000  
             Szekesfehervar  0.000000  
             Veszprem        0.000000  
...                               ...  
Zalaegerszeg Szeged         -1.079081  
             Szekesfehervar -0.351365  
             Szekszard      -0.559632  
             Tatabanya       0.000000  
             Veszprem        0.000000  

[171 rows x 11 columns]

In [156]:
spatial_timelines.cumsum(axis=1)[spatial_timelines.cumsum(axis=1)[2012]<-0.3]

year                         2002  2003      2004      2005      2006  \
source       target                                                     
Bekescsaba   Miskolc          0.0   0.0 -0.108609 -0.108609 -0.558268   
Budapest     Debrecen         0.0   0.0  0.000000  0.000000 -0.424674   
             Nyiregyhaza      0.0   0.0  0.000000  0.000000 -0.042300   
             Pecs             0.0   0.0  0.000000  0.000000 -0.195353   
             Szeged           0.0   0.0  0.000000  0.000000 -0.587912   
...                           ...   ...       ...       ...       ...   
Zalaegerszeg Nyiregyhaza      0.0   0.0  0.000000  0.000000 -0.278981   
             Salgotarjan      0.0   0.0  0.000000  0.000000 -0.236682   
             Szeged           0.0   0.0  0.000000  0.000000 -0.824858   
             Szekesfehervar   0.0   0.0  0.000000  0.000000 -0.146245   
             Szekszard        0.0   0.0  0.000000  0.000000 -0.358691   

year                             2007      2008      2009      2010      2011  \
source       target                                                             
Bekescsaba   Miskolc        -0.558268 -0.558268 -0.558268 -0.558268 -0.558268   
Budapest     Debrecen       -0.424674 -0.424674 -0.424674 -0.424674 -0.424674   
             Nyiregyhaza    -0.571295 -0.571295 -0.571295 -0.571295 -0.571295   
             Pecs           -0.234014 -0.275446 -0.275446 -0.505723 -0.505723   
             Szeged         -0.587912 -0.587912 -0.587912 -0.587912 -0.639039   
...                               ...       ...       ...       ...       ...   
Zalaegerszeg Nyiregyhaza    -1.013096 -1.013096 -1.013096 -1.013096 -1.013096   
             Salgotarjan    -0.441801 -0.441801 -0.441801 -0.441801 -0.441801   
             Szeged         -1.032718 -1.032718 -1.032718 -1.032718 -1.079081   
             Szekesfehervar -0.351365 -0.351365 -0.351365 -0.351365 -0.351365   
             Szekszard      -0.559632 -0.559632 -0.559632 -0.559632 -0.559632   

year                             2012  
source       target                    
Bekescsaba   Miskolc        -0.558268  
Budapest     Debrecen       -0.424674  
             Nyiregyhaza    -0.571295  
             Pecs           -0.505723  
             Szeged         -0.639039  
...                               ...  
Zalaegerszeg Nyiregyhaza    -1.013096  
             Salgotarjan    -0.441801  
             Szeged         -1.079081  
             Szekesfehervar -0.351365  
             Szekszard      -0.559632  

[67 rows x 11 columns]